In [1]:
import pandas as pd

import time
import numpy as np
import math
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch import nn

import datetime as dt

from QLSTM import SequenceDataset
from QLSTM_Noisy import SequenceDataset

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

from QLSTM import QShallowRegressionLSTM

# Calculate the RMSE for the train and test data
from sklearn.metrics import mean_squared_error

import wrds

# Connect to WRDS
db = wrds.Connection()

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


Enter your WRDS username [ch0002techvc]: ch0002ic
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [ ]:
# Define CRSP table and columns of interest
table = "dsf"  # Example daily stock file table
columns = ["permno", "date", "bidlo", "askhi", "prc", "vol"]

# Query CRSP table for the required data
query = f"""
    SELECT {', '.join(columns)}
    FROM crsp.{table}
    WHERE date BETWEEN '2021-12-31' AND '2022-12-31'
"""
crsp_data = db.raw_sql(query)

# Process the data: Extract 'Open', 'High', 'Low', 'Close' prices
# For CRSP, these are approximations: 'bidlo' as Low, 'askhi' as High, and 'prc' as Close
crsp_data.rename(columns={'bidlo': 'Low', 'askhi': 'High', 'prc': 'Close'}, inplace=True)

# Note: CRSP does not explicitly provide 'Open' prices in its dataset
# Shift 'Close' to create 'Open' prices for each permno
crsp_data['Open'] = crsp_data.groupby('permno')['Close'].shift(1)

# Drop the first row for each permno as it will have NaN 'Open' prices
crsp_data = crsp_data.dropna(subset=['Open'])
crsp_data.to_csv("crsp_data.csv", index=False)